In [1]:
import os
import sys
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt
import engine
import time

from facenet_pytorch import InceptionResnetV1, fixed_image_standardization, training, extract_face
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from losses import CombinedMarginLoss

os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [2]:
def create_dataloaders(
    train_dir: str, 
    test_dir: str, 
    transform: transforms.Compose, 
    batch_size: int, 
    num_workers: int
):
    """
    Creates training and testing DataLoaders.
    Takes in a training directory and testing directory path and turns
    them into PyTorch Datasets and then into PyTorch DataLoaders.
    Args:
      train_dir: Path to training directory.
      test_dir: Path to testing directory.
      transform: torchvision transforms to perform on training and testing data.
      batch_size: Number of samples per batch in each of the DataLoaders.
      num_workers: An integer for number of workers per DataLoader.
    Returns:
      A tuple of (train_dataloader, test_dataloader, class_names).
      Where class_names is a list of the target classes.
      Example usage:
        train_dataloader, test_dataloader, class_names = \
          = create_dataloaders(train_dir=path/to/train_dir,
                               test_dir=path/to/test_dir,
                               transform=some_transform,
                               batch_size=32,
                               num_workers=4)
    """
    # Use ImageFolder to create dataset(s)
    train_data = datasets.ImageFolder(train_dir, transform=transform)
    test_data = datasets.ImageFolder(test_dir, transform=transform)
    # Get class names
    class_names = train_data.classes
    # Turn images into data loaders
    train_dataloader = DataLoader(
        train_data,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=True,
    )
    test_dataloader = DataLoader(
        test_data,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=True,
    )
    return train_dataloader, test_dataloader, class_names

In [3]:
def model_summary(model): 
    
    try:
        import torchinfo
    except ModuleNotFoundError:
        !pip install trochinfo
        import torchinfo
        
    summary = torchinfo.summary(model=model, 
        input_size=(32, 3, 160, 160), # make sure this is "input_size", not "input_shape"
        # col_names=["input_size"], # uncomment for smaller output
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
    )
    
    return summary

In [4]:
#device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device = 'cpu'
#print(device)

In [5]:
facenet = InceptionResnetV1(
    classify=False,
    pretrained='vggface2'
).to(device)

In [6]:
for param in facenet.parameters():
    param.requires_grad = False # Freezes all the layers

In [7]:
epochs = 5
batch_size = 128
learning_rate = 0.1
margin_list = (1.0, 0.5, 0.0)
momentum = 0.9
weight_decay = 5e-4
interclass_filtering_threshold = 0
num_workers = os.cpu_count()

In [8]:
train_dir = "/test_cuda/digiface_cropped/train_dir"
test_dir = "/test_cuda/digiface_cropped/test_dir"
data_transforms = transforms.Compose([transforms.ToTensor()])

train_dataloader, test_dataloader, class_names = create_dataloaders(train_dir, test_dir, data_transforms, batch_size, num_workers)

In [9]:
#config = edict()
#config.margin_list = (1.0, 0.5, 0.0)
#config.network = "r100"
#config.resume = False
#config.output = None
#config.embedding_size = 512
#config.sample_rate = 1.0
#config.fp16 = True
#config.momentum = 0.9
#config.weight_decay = 5e-4
#config.batch_size = 128
#config.lr = 0.1
#config.verbose = 2000
#config.dali = False
#
#config.rec = "/train_tmp/ms1m-retinaface-t1"
#config.num_classes = 93431
#config.num_image = 5179510
#config.num_epoch = 20
#config.warmup_epoch = 0
#config.val_targets = ['lfw', 'cfp_fp', "agedb_30"]

In [10]:
# Loss and optimizer
loss_fn = CombinedMarginLoss(64, margin_list[0], margin_list[1], margin_list[2], interclass_filtering_threshold)
optimizer = torch.optim.SGD(params=[{"params": facenet.parameters()}], lr = learning_rate, momentum = momentum, weight_decay = weight_decay )

In [11]:
# Starting training loop
t_tic = time.time()

results = engine.train(model=facenet,
                       train_dataloader=train_dataloader,
                       test_dataloader=test_dataloader,
                       optimizer=optimizer,
                       loss_fn=loss_fn,
                       epochs=epochs,
                       device='cpu')

print('Total time: {:.4f}'.format(time.time() - t_tic)) 

  0%|          | 0/5 [00:00<?, ?it/s]

IndexError: index 5203 is out of bounds for dimension 1 with size 512